In [1]:
import os
from Crypto.Cipher import ChaCha20

# --- Stream cipher helpers ---
def encrypt_message(key, plaintext, nonce=None):
    """
    Encrypt a message with ChaCha20.
    Returns (ciphertext, nonce).
    If no nonce is given, a random 8-byte nonce is generated.
    """
    if nonce is None:
        nonce = os.urandom(8)
    cipher = ChaCha20.new(key=key, nonce=nonce)
    ciphertext = cipher.encrypt(plaintext)
    return ciphertext, nonce

def decrypt_message(key, ciphertext, nonce):
    """
    Decrypt a message with ChaCha20 given key and nonce.
    """
    cipher = ChaCha20.new(key=key, nonce=nonce)
    return cipher.decrypt(ciphertext)


# --- Simulated chat ---
print("=== Alice ↔ Bob Secure Chat Simulation ===")

# Shared symmetric key (e.g. derived via Diffie-Hellman beforehand)
shared_key = os.urandom(32)

# Conversation (sequence of messages)
conversation = [
    ("Alice", b"Hi Bob, how are you today?"),
    ("Bob",   b"Hey Alice, I am fine, thanks! What about you?"),
    ("Alice", b"I'm great. Want to meet later?"),
    ("Bob",   b"Sure, let's meet at 6 PM."),
]

# --- Messaging loop ---
transcript = []

for sender, msg in conversation:
    # Encrypt with a fresh nonce
    ciphertext, nonce = encrypt_message(shared_key, msg)
    
    # Store what was "sent" (header + ciphertext)
    transcript.append((sender, msg, ciphertext, nonce))
    
    # Show what goes over the network
    print(f"\n[{sender}] sends:")
    print("  Nonce (header):", nonce.hex())
    print("  Ciphertext:    ", ciphertext.hex())
    
    # Receiver decrypts
    decrypted = decrypt_message(shared_key, ciphertext, nonce)
    print("  Receiver decrypts ->", decrypted.decode())


# --- Security failure: nonce reuse demo ---
print("\n=== Nonce reuse attack (two-time pad) ===")

m1 = b"Hello, this is Alice"
m2 = b"I need help from you"

# Bad practice: same nonce used for both
bad_nonce = os.urandom(8)

c1, _ = encrypt_message(shared_key, m1, nonce=bad_nonce)
c2, _ = encrypt_message(shared_key, m2, nonce=bad_nonce)

# Attacker computes XOR of ciphertexts
xor_ct = bytes([a ^ b for a, b in zip(c1, c2)])
xor_pt = bytes([a ^ b for a, b in zip(m1, m2)])

print("C1 XOR C2:", xor_ct)
print("M1 XOR M2:", xor_pt)

print("\n>>> Notice: C1 XOR C2 = M1 XOR M2. Reusing the same nonce leaks information!")


=== Alice ↔ Bob Secure Chat Simulation ===

[Alice] sends:
  Nonce (header): f52f2496556014f5
  Ciphertext:     47b72b0871b57da74d5641b8b302b5f3a58dad5d9acbaa669ce9
  Receiver decrypts -> Hi Bob, how are you today?

[Bob] sends:
  Nonce (header): 80af3e4d1987bb2c
  Ciphertext:     facd489c71cb45fff88e555eccc475bf5fccd4977cb6b5204278d479f6fedd71b9307428a01db29d0f1b525ee3
  Receiver decrypts -> Hey Alice, I am fine, thanks! What about you?

[Alice] sends:
  Nonce (header): cdfa115c3a42ae17
  Ciphertext:     a738338303ab555076d467b1649467853df12b0df8aeeaa91d05d543009c
  Receiver decrypts -> I'm great. Want to meet later?

[Bob] sends:
  Nonce (header): 10bfb724419b403c
  Ciphertext:     58d63224d3531c30d3d3b02e1398ce4e8510924849ad922886
  Receiver decrypts -> Sure, let's meet at 6 PM.

=== Nonce reuse attack (two-time pad) ===
C1 XOR C2: b'\x01E\x02\t\nH\x00\x1c\r\x05\x03\x00\x0f\x01O,L\x10\x0c\x10'
M1 XOR M2: b'\x01E\x02\t\nH\x00\x1c\r\x05\x03\x00\x0f\x01O,L\x10\x0c\x10'

>>> Notice: C1 

---

### Recap of how a stream cipher works

Encryption is:

$$C = M \oplus KS$$

where

* (M) = plaintext message (bits),
* (KS) = keystream generated from (Key, Nonce, Counter),
* (C) = ciphertext.

Decryption is the same operation:

$$M = C \oplus KS$$

---

###  Problem when nonce is reused

Suppose we encrypt **two different messages** (M_1, M_2) with the **same keystream** (i.e. same key and same nonce):
$$ C_1 = M_1 \oplus KS$$

$$C_2 = M_2 \oplus KS$$

Now an attacker computes:

$$C_1 \oplus C_2 = (M_1 \oplus KS) \oplus (M_2 \oplus KS)$$

Since $$KS \oplus KS = 0$$

we have that:

$$C_1 \oplus C_2 = M_1 \oplus M_2$$

So, in certain sense, we know a certain relationship between both messages...!

---

## Why is this dangerous?

* Even though the attacker doesn’t know (M_1) or (M_2) individually, the relation between them is now exposed.
* If attacker knows (or can guess) part of one message, they immediately recover the corresponding part of the other.
* This is called a **two-time pad attack** (like the one-time pad but used more than once).

---

## Example

Say Alice encrypts:

* (M_1 = \text{"HELLO"})
* (M_2 = \text{"WORLD"})

with the same keystream `KS`.

An attacker sees (C_1, C_2) and computes:

$$
C_1 \oplus C_2 = \text{"HELLO"} \oplus \text{"WORLD"}
$$

This produces a weird-looking XOR, but if the attacker can guess that one message starts with `"HELLO"`, they can recover `"WORLD"` entirely.

That’s how World War II codebreakers (e.g. Venona project) cracked reused OTPs.

---

## Real-world impact

* If WhatsApp/Signal/etc. ever reused a nonce with the same key, **every message would leak information about the others**.
* In practice, it can allow attackers to:

  * Recover entire plaintexts (if part is guessable, like headers, file formats, “HTTP/1.1”, etc.).
  * Mount statistical or crib-dragging attacks.
  * Break confidentiality completely.

---

## Corollary: Stream cipher = safe only if keystream is never reused.
* That’s why implementations enforce **unique nonces** (or counters) for every encryption under the same key.
* Protocols like TLS, Signal’s Double Ratchet, etc. are designed specifically to guarantee that nonce reuse doesn’t happen.

<font color='blue'> Assume we know the message $m_1$ starts with: "Hello, this is". Then we can recover that part of $m_2$ as follows:</font> 

In [2]:
# Cell 4 — Known-plaintext attack demo
# Suppose attacker somehow knows that m1 starts with b"Subject: "
known_prefix = b"Hello, this is"
kp_len = len(known_prefix)
recovered_part_of_m2 = bytes(k ^ c for k, c in zip(known_prefix, xor_ct[:kp_len]))
print("Known prefix of M1:", known_prefix)
print("Recovered corresponding bytes of M2:", recovered_part_of_m2)
print("Interpreted as text (maybe):", recovered_part_of_m2.decode(errors='replace'))


Known prefix of M1: b'Hello, this is'
Recovered corresponding bytes of M2: b'I need help fr'
Interpreted as text (maybe): I need help fr
